In [1]:
# Required for test cases
def is_overridden(func):
    obj = func.__self__
    parent_methods = getattr(super(type(obj), obj), func.__name__)

    return func.__func__ != parent_methods.__func__

Class `Account` is defined as an abstract base class (ABC).
- `__init__(self,  name, balance):` takes two parameters `name`, which represents the account holder's name, and `balance`, which represents the initial balance of the account.
    - These parameters are set to the instance variables `name` and `balance` respectively
    - The `balance` stores the current balance of the account. It's a private attribute and can be accessed through the balance property. The `__balance` attribute must be ensured to remain non-negative whenever it is updated.
- `deposit(self, amount)` and `withdraw(self, amount):` handle transactions, raising ValueError if the input amount is negative or exceeds the available balance, respectively.
- `show(self)` returns the message of the account holder's name and current balance.
- `calculate_interest(self)`, marked as abstract, calculates interest for the account.

**Q1:** Implement getter and setter for the `__balance` attribute

In [2]:
from abc import ABC, abstractmethod
class Account(ABC):
    # YOUR CODE HERE
    def __init__(self, name, balance):
        self.name = name
        self.balance = balance

    @property
    def balance(self):
        return self.__balance

    @balance.setter
    def balance(self, balance):
        if balance < 0:
            raise ValueError('Balance must be non-negative')
        self.__balance = balance

    def deposit(self, amount):
        if amount < 0:
            raise ValueError('Deposit amount must be non-negative')
        self.balance += amount
        return self.balance

    def withdraw(self, amount):
        if amount < 0:
            raise ValueError('Withdrawal amount must be non-negative')
        if amount > self.balance:
            raise ValueError('Withdrawal amount exceeds available balance')
        self.balance -= amount
        return self.balance

    def show(self):
        txt = ' Name: ' + self.name + '\n'
        txt += ' Balance: ' + str(self.balance) + '\n'
        return txt

    @abstractmethod
    def calculate_interest(self):
        raise NotImplementedError

**Q2:** Complete the following subclasses that extend the functionality of the base `Account` class, providing specialized behavior for different types of financial accounts as follows.

The `SavingsAccount` class is a subclass of the `Account` class, representing a savings account.
- It initializes with the account holder's name (`name`), initial balance (`balance`), and the interest rate (`interest_rate`) for calculating interest.
- The `calculate_interest(self)` method calculates the interest earned based on the account balance and the specified interest rate.

In [3]:
class SavingsAccount(Account):
    # YOUR CODE HERE
    def __init__(self, name, balance, interest_rate):
        super().__init__(name, balance)
        self.interest_rate = interest_rate
    
    @property
    def interest_rate(self):
        return self._interest_rate

    @interest_rate.setter
    def interest_rate(self, interest_rate):
        self._interest_rate = interest_rate

    def calculate_interest(self):
        return self._interest_rate * self.balance / 100

In [4]:
def test_savings_account():
    # Create a savings account with initial balance and interest rate
    acc = SavingsAccount('John', 1000, 2.5)

    # Test method overriding
    assert is_overridden(acc.deposit) == False
    assert is_overridden(acc.withdraw) == False
    assert is_overridden(acc.show) == False
    assert is_overridden(acc.calculate_interest) == True

    # Test initialization
    assert acc.name == 'John'
    assert acc.balance == 1000
    assert acc._interest_rate == 2.5

    # Test calculate interest method
    interest = acc.calculate_interest()
    assert interest == 25.0  # (1000 * 2.5) / 100 = 25.0

    # Test the initialization with negative balance
    try:
        SavingsAccount('John', -1000, 2.5)
    except Exception:
        pass
    else:
        raise AssertionError("Expected ValueError not raised")

    print("All test cases passed!")

# Run test cases
test_savings_account()

All test cases passed!


The `CheckingAccount` class is also a subclass of the `Account` class, representing a checking account.
- It initializes with the account holder's name (`name`), initial balance (`balance`), and the overdraft limit (`overdraft_limit`), which defines the maximum negative balance allowed.
- The `calculate_interest(self)` method returns 0, indicating that checking accounts typically do not earn interest.
- The `deposit(self, amount)` allows depositing a certain amount into the checking account.
    - If the overdraft is not utilized (i.e., `_overdraft` == 0), the amount is directly deposited into the account balance using the superclass's `deposit` method. The overdraft remains unaffected.
    - If there is an overdraft (i.e., `_overdraft` > 0), the deposited amount is first used to cover the overdraft.
        - If the deposited amount is greater than or equal to the overdraft, the overdraft is fully covered, and the remaining amount is added to the account balance.
        - If the deposited amount is less than the overdraft, the deposited amount is subtracted from the overdraft, and the balance remains zero.
    - Finally, the method returns a tuple containing the updated balance and overdraft.

- The `withdraw(self, amount)` method allows withdrawing money from the account, checking against the available balance, the current overdraft, and the overdraft limit.
    - If the requested withdrawal amount is less than or equal to the current balance, the withdrawal is straightforward and is handled by the superclass's withdraw method, which updates the balance accordingly.
    - Otherwise, the requested amount is further chekced against the available funds (current balance + overdraft limit - current overdraft).
        - If the requested withdrawal amount exceeds the available funds, a ValueError is raised with an appropriate message.
        - If the requested withdrawal amount can be covered by the available funds, the balance is set to zero, and the remaining amount needed to cover the withdrawal is added to the overdraft.
    - Finally, the method returns a tuple containing the updated balance and overdraft.

- The `show` method returns the message of the account holder's name, balance, and overdraft.

In [5]:
class CheckingAccount(Account):
    # YOUR CODE HERE
    def __init__(self, name, balance, overdraft_limit = 0):
        super().__init__(name, balance)
        self.overdraft_limit = overdraft_limit
        self.overdraft = 0
    
    @property
    def overdraft_limit(self):
        return self._overdraft_limit
    
    @overdraft_limit.setter
    def overdraft_limit(self, overdraft_limit):
        self._overdraft_limit = overdraft_limit
    
    @property
    def overdraft(self):
        return self._overdraft
    
    @overdraft.setter
    def overdraft(self, overdraft):
        if overdraft > self.overdraft_limit:
            raise ValueError('Overdraft exceeds overdraft limit')
        self._overdraft = overdraft

    def calculate_interest(self):
        return 0

    def deposit(self, amount):
        if self.overdraft == 0:
            super().deposit(amount)
        elif self.overdraft > 0:
            if amount >= self.overdraft:
                amount = amount - self.overdraft
                self.overdraft = 0
                self.balance += amount
            else:
                self.overdraft = self.overdraft - amount
                self.balance = 0
        return (self.balance, self.overdraft)
    
    def withdraw(self, amount):
        if amount <= self.balance:
            super().withdraw(amount)
        else:
            available_fund = self.balance + self.overdraft_limit - self.overdraft
            if amount > available_fund:
                raise ValueError('Withdrawal amount exceeds available balance')
            else:
                self.overdraft += amount - self.balance
                self.balance = 0
        return (self.balance, self.overdraft)
    
    def show(self):
        txt = ' Name: ' + self.name + '\n'
        txt += ' Balance: ' + str(self.balance) + '\n'
        txt += ' Overdraft: ' + str(self.overdraft) + '\n'
        return txt

In [6]:
def test_checking_account():
    # Create a CheckingAccount object
    acc = CheckingAccount('Alice', 1000, 500)

    # Test method overriding
    assert is_overridden(acc.deposit) == True
    assert is_overridden(acc.withdraw) == True
    assert is_overridden(acc.show) == True
    assert is_overridden(acc.calculate_interest) == True

    # Test initialization
    assert acc.name == 'Alice'
    assert acc.balance == 1000
    assert acc._overdraft_limit == 500
    assert acc._overdraft == 0

    # Test deposit method
    acc.deposit(200)
    assert acc.balance == 1200
    assert acc._overdraft == 0

    # Test withdraw method
    acc.withdraw(300)
    assert acc.balance == 900
    assert acc._overdraft == 0

    # Test overdraft functionality
    acc.withdraw(1300)
    assert acc.balance == 0
    assert acc._overdraft == 400

    # Test deposit to reduce overdraft
    acc.deposit(40)
    assert acc.balance == 0
    assert acc._overdraft == 360

    # Test deposit to cover overdraft
    acc.deposit(400)
    assert acc.balance == 40
    assert acc._overdraft == 0

    # Test withdrawal exceeding balance and overdraft limit
    try:
        acc.withdraw(600)
    except ValueError as e:
        pass
    else:
        raise AssertionError("Expected ValueError not raised")

    print("All test cases passed!")

# Run test cases
test_checking_account()

All test cases passed!


The `PremiumFeatures` class provides additional features for premium accounts.

- `__init__(self, bonus_interest_rate):`
    - Initializes a new instance of the PremiumFeatures class with a bonus interest rate.
    - Parameters:
        - `bonus_interest_rate`: The bonus interest rate for the premium account.
- `calcuate_bonus_interest(self, balance):`
    - Calculates the bonus interest based on the balance of the account.
    - Parameters:
        - `balance`: The current balance of the account.
    - Returns the calculated bonus interest.

In [7]:
class PremiumFeatures:
    def __init__(self, bonus_interest_rate):
        self._bonus_interest_rate = bonus_interest_rate

    def calcuate_bonus_interest(self, balance):
        bonus_interest = balance * self._bonus_interest_rate / 100
        return bonus_interest

**Q3:** Create the subclass, `PremiumAccount` which represents a premium savings account with additional bonus interest features (multiple inheritance). The `PremiumAccount` class combines functionalities from the `SavingsAccount` class and the `PremiumFeatures` class to offer a comprehensive solution for managing premium accounts.

- `__init__(self, name, balance, interest_rate, bonus_interest_rate):`
    - Initializes a new instance of the PremiumAccount class.
    - Calls the constructors of parent classes (SavingsAccount and PremiumFeatures) to initialize attributes.
    - Parameters:
        - `name`: The name of the account holder.
        - `balance`: The initial balance of the account.
        - `interest_rate`: The base interest rate for calculating interest.
        - `bonus_interest_rate`: The rate of bonus interest applicable to the account.
- `calculate_interest(self):`
    - Calculates the total interest earned by the premium account.
    - Calls the `calculate_interest` method of the `SavingsAccount` superclass to calculate base interest.
    - Calls the `calcuate_bonus_interest` method of the `PremiumFeatures` superclass to calculate bonus interest.
    - Returns the sum of base interest and bonus interest as the total interest earned.

In [8]:
# YOUR CODE HERE
class PremiumAccount(SavingsAccount, PremiumFeatures):
    def __init__(self, name, balance, interest_rate, bonus_interest_rate):
        SavingsAccount.__init__(self, name, balance, interest_rate)
        PremiumFeatures.__init__(self, bonus_interest_rate)
    
    def calculate_interest(self):
        interest = super().calculate_interest()
        bonus_interest = self.calcuate_bonus_interest(self.balance)
        return interest + bonus_interest

In [9]:
def test_premium_account():
    acc = PremiumAccount("Dan", 1000, 2.0, 0.5)

    # Test method overriding
    assert is_overridden(acc.deposit) == False
    assert is_overridden(acc.withdraw) == False
    assert is_overridden(acc.show) == False
    assert is_overridden(acc.calculate_interest) == True

    # Test initialization
    assert acc.name == "Dan"
    assert acc.balance == 1000
    assert acc._interest_rate == 2.0
    assert acc._bonus_interest_rate == 0.5

    # Test calculate_interest
    # Total interest = 20 + 5 = 25
    assert acc.calculate_interest() == 25

    # Test withdraw
    acc.withdraw(200)
    assert acc.balance == 800

    # Test deposit
    acc.deposit(500)
    assert acc.balance == 1300

    print("All test cases passed!")

test_premium_account()

All test cases passed!


The `Bank` class represents a simplified bank entity that manages accounts.

- Attributes:

    - `_accounts`: A dictionary to store Account objects.
    - `_next_account_number`: An integer representing the next available account number.
- Methods:

    - `__init__(self):`
        - Initializes `_accounts` as an empty dictionary to store customer accounts and `_next_account_number` to create account id for each customer account starting at 0.
    - `create_account(self, name, initial_amount, account_type='savings', **kwargs):`
        - Parameters:
            - `name`: The name of the account holder.
            - `initial_amount`: The initial amount to deposit in the account.
            - `account_type`: The type of account to create (default is 'savings').
            - `**kwargs`: Additional keyword arguments specific to each account type (e.g., interest rate, overdraft limit).
        - Creates a new account and adds it to the bank's accounts.
        - Returns the new account number.
    - `all_add_interest(self):`
        - Calculates interest for each account and deposits it into its corresponding account.
    - `all_show(self):`
        - Prints details i.e. account number, account holder's name, and balance for each account.

**Q4:** Complete the `all_add_interest(self)` and `all_show(self)` methods in the `Bank` class.

In [10]:
class Bank:
    def __init__(self):
        self._accounts = {}  # Dictionary to store Account objects
        self._next_account_number = 0

    def create_account(self, name, initial_amount, account_type='savings', **kwargs):
        if account_type == 'savings':
            interest_rate = kwargs.get('interest_rate', 1.5)  # Default interest rate for savings account
            account = SavingsAccount(name, initial_amount, interest_rate)
        elif account_type == 'checking':
            overdraft_limit = kwargs.get('overdraft_limit', 1000)  # Default overdraft limit for checking account
            account = CheckingAccount(name, initial_amount, overdraft_limit)
        elif account_type == 'premium':
            interest_rate = kwargs.get('interest_rate', 1.5)  # Default interest rate for premium account
            overdraft_limit = kwargs.get('overdraft_limit', 1000)  # Default overdraft limit for premium account
            account = PremiumAccount(name, initial_amount, interest_rate, overdraft_limit)
        else:
            raise ValueError("Invalid account type. Creating savings account by default.")

        new_account_number = self._next_account_number
        self._accounts[new_account_number] = account
        self._next_account_number += 1
        return new_account_number

    def balance(self, account_number):
        account = self._accounts.get(account_number)
        if account is None:
            raise KeyError('Account not found')
        return account.balance

    def all_add_interest(self):
        # YOUR CODE HERE
        for account_number, account in self._accounts.items():
            interest = account.calculate_interest()
            account.deposit(interest)

    def all_show(self):
        display_text = ''
        for account_number, account in self._accounts.items():
            display_text += f' Account: {account_number}\n{account.show()}'
        print(display_text)
        return display_text

In [11]:
def test_bank():
    # Initialize a Bank object
    oBank = Bank()

    # Create various accounts
    marys_account_number = oBank.create_account('Mary', 12345, account_type='savings', interest_rate=2.0)
    bobs_account_number = oBank.create_account('Bob', 5000, account_type='checking', overdraft_limit=1000)
    anns_account_number = oBank.create_account('Ann', 20000, account_type='premium', interest_rate=3.0, overdraft_limit=5000)

    # Add interest to all accounts
    oBank.all_add_interest()
    assert oBank.balance(marys_account_number) == 12591.9
    assert oBank.balance(bobs_account_number) == 5000
    assert oBank.balance(anns_account_number) == 1020600.0

    # Show all accounts
    assert oBank.all_show() == " Account: 0\n Name: Mary\n Balance: 12591.9\n Account: 1\n Name: Bob\n Balance: 5000\n Overdraft: 0\n Account: 2\n Name: Ann\n Balance: 1020600.0\n"

    print("All test cases passed!")

test_bank()


 Account: 0
 Name: Mary
 Balance: 12591.9
 Account: 1
 Name: Bob
 Balance: 5000
 Overdraft: 0
 Account: 2
 Name: Ann
 Balance: 1020600.0

All test cases passed!
